In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_path = "../dataset/processed_images/train"
val_path = "../dataset/processed_images/validation"
test_path = "../dataset/processed_images/test"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 12493 images belonging to 5 classes.
Found 2248 images belonging to 5 classes.


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam

In [2]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False  # Freeze base layers

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [3]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(5, activation='softmax')  # 5 rice classes
])

In [4]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=3
)

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 253s 646ms/step - accuracy: 0.9997 - loss: 7.7774e-04 - val_accuracy: 1.0000 - val_loss: 2.8845e-04
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 263s 672ms/step - accuracy: 0.9996 - loss: 0.0010 - val_accuracy: 0.9996 - val_loss: 5.3356e-04
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 259s 662ms/step - accuracy: 0.9994 - loss: 0.0012 - val_accuracy: 0.9996 - val_loss: 7.6815e-04


In [12]:
import pickle
import os

os.makedirs("../results", exist_ok=True)

with open("../results/training_history.pkl", "wb") as f:
    pickle.dump(history.history, f)

print("History saved successfully.")

History saved successfully.


In [9]:
model.save("../models/mobilenet_model.h5")